# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

data0 = pd.read_csv('/datasets/geo_data_0.csv')
data1 = pd.read_csv('/datasets/geo_data_1.csv')
data2 = pd.read_csv('/datasets/geo_data_2.csv')
data1

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [2]:
data0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [3]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Количество данных одинаково по 100000 объектов. Признак "id" нам не нужен для обучения модели. Целевым признаком будет "product". Разделим данные на обучающую и валидационную.

In [5]:
df_0 = data0.drop(['id'], axis=1)
df_1 = data1.drop(['id'], axis=1)
df_2 = data2.drop(['id'], axis=1)
df_0

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...
99995,0.971957,0.370953,6.075346,110.744026
99996,1.392429,-0.382606,1.273912,122.346843
99997,1.029585,0.018787,-1.348308,64.375443
99998,0.998163,-0.528582,1.583869,74.040764


In [6]:
# функция разделения данных
def split(df):
    target = df['product']
    features = df.drop(['product'], axis=1)
    features_train, features_valid, target_train, target_valid = train_test_split(\
    features, target, test_size=0.25, random_state=12345)
    return features_train, features_valid, target_train, target_valid
features_train_0, features_valid_0, target_train_0, target_valid_0 = split(df_0)
target_train_0.mean(), target_valid_0.mean()

(92.64046775305692, 92.07859674082927)

In [7]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = split(df_1)
target_train_1.mean(), target_valid_1.mean()

(68.85895465854666, 68.72313602435997)

In [8]:
features_train_2, features_valid_2, target_train_2, target_valid_2 = split(df_2)
target_train_2.mean(), target_valid_2.mean()

(95.03858906371522, 94.88423280885438)

На первый взгляд во втором регионе среднее количество запасов меньше чем в остальных. Большее среднее количество продукта в третьем регионе.

## Обучение и проверка модели

In [9]:
# функция обучения модели
def LgR(features_train, target_train, features_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    return predicted_valid


Теперь расчитаем предполагаемую выручку по все регионам. Так же вычислим разброс показателей от среднего значения.

In [10]:
predicted_valid_0 = LgR(features_train_0, target_train_0, features_valid_0)
predicted_valid_1 = LgR(features_train_1, target_train_1, features_valid_1)
predicted_valid_2 = LgR(features_train_2, target_train_2, features_valid_2)
# расчет RMSE и MAE
rmse = []
mae = []
predicted_all = [predicted_valid_0,predicted_valid_1,predicted_valid_2]
target_all = [target_valid_0,target_valid_1,target_valid_2]
for predicted, target in zip(predicted_all, target_all):
    rmse.append(mean_squared_error(target, predicted)**0.5)
    mae.append(mean_absolute_error(target, predicted))

rmse, mae

([37.5794217150813, 0.893099286775617, 40.02970873393434],
 [30.919600777151313, 0.7187662442124758, 32.792652105481814])

Видно что в первом и третьем регионе модель имеет большие отклонения, второй регион показывает меньший разброс.

## Подготовка к расчёту прибыли

In [11]:
# создадим переменные
number = 500
count = 200
budget = 10000
price = 0.45

Расчитаем достаточное количество сырья для безубыточности

In [12]:
product_min = budget/price/count
product_min

111.11111111111111

добыча больше 111 тысяч на точку нам обеспечит неубыточность региона, тогда необходимо оценить среднее количество продукта по регионам

In [13]:
df_all = [df_0, df_1, df_2]
for i in range(len(df_all)):
    print('среднее всего региона', df_all[i]['product'].mean())
    print('среднее 200 лучших   ',df_all[i]['product'].sort_values(ascending=False).head(200).mean())

среднее всего региона 92.50000000000001
среднее 200 лучших    184.83373964536008
среднее всего региона 68.82500000000002
среднее 200 лучших    137.9454077409057
среднее всего региона 95.00000000000004
среднее 200 лучших    189.55147698176646


Первый и третий регион обладают большим средним запасом нефти и скорее всего среди них будет достаточное количество скважин для разработок в пределаж бюджета. А из 200 лучших точек средние объемы превышают минимальный показатель.

## Расчёт прибыли и рисков 

Расчитаем потенциальную прибыль с 200 лучших точек.

In [14]:
# функция расчета прибыли
def profit(target_valid, predicted_valid, count):
    predict_best = predicted_valid.sort_values(ascending=False)
    volume = target_valid[predict_best.index][:count]
    profit = (price * volume.sum() - budget)/1000
    return profit
for target, predicted in zip(target_all, predicted_all):
    target=target.reset_index(drop=True)
    predicted=pd.Series(predicted)
    print(profit(target, predicted, 200))


3.32082604313985
2.415086696681512
2.7103499635998323


Выручка по регионам не сильно отличается. Но у нас не все данные в выборке. Для верного расчета надо взять весь перечень точек.

In [15]:
features_0 = df_0.drop(['product'], axis=1)
features_1 = df_1.drop(['product'], axis=1)
features_2 = df_2.drop(['product'], axis=1)
target_0 = df_0['product']
target_1 = df_1['product']
target_2 = df_2['product']

Расчитаем предсказания

In [16]:
predict_0 = LgR(features_train_0, target_train_0, features_0)
predict_0=pd.Series(predict_0)
predict_1 = LgR(features_train_1, target_train_1, features_1)
predict_1=pd.Series(predict_1)
predict_2 = LgR(features_train_2, target_train_2, features_2)
predict_2=pd.Series(predict_2)
predict_0

0         95.461973
1        116.227394
2         88.750254
3         95.419237
4        114.138969
            ...    
99995    116.174856
99996     96.652519
99997     72.401340
99998     99.337548
99999    125.684762
Length: 100000, dtype: float64

Расчитаем прибыль на всей выберке

In [17]:
profit_0 = profit(target_0, predict_0, 200)
profit_1 = profit(target_1, predict_1, 200)
profit_2 = profit(target_2, predict_2, 200)
print('прибыль первого региона', profit_0)
print('прибыль второго региона', profit_1)
print('прибыль третьего региона', profit_2)

прибыль первого региона 3.4941041921120157
прибыль второго региона 2.415086696681512
прибыль третьего региона 2.571410631960929


Потенциальная прибыль первого региона выше остальных.

Расчитаем риски неполучения прибыли по регионам.

In [18]:
state = np.random.RandomState(12345)
# функция расчета рисков
def risks(target, predictions):
    values=[]
    for i in range(1000):
        target_subsample=target.sample(n=500, replace=True, random_state=state)
        probs_subsample=predictions[target_subsample.index]
        values.append(profit(target_subsample, probs_subsample, 200))
    values = pd.Series(values)
    mean = values.mean()
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    risk = (values[values<0].count()/len(values))*100
    print('Средняя прогнозируемая прибыль равна', mean)
    print('С вероятностью 95% прибыль будет от', lower, 'до', upper, 'млрд. руб.')
    print('риск убытка', risk)

In [19]:
print('Прогноз для первого региона')
risks(target_0, predict_0)

Прогноз для первого региона
Средняя прогнозируемая прибыль равна 0.4310923134956579
С вероятностью 95% прибыль будет от -0.07054176729685392 до 0.9790196590485124 млрд. руб.
риск убытка 5.1


In [20]:
print('Прогноз для второго региона')
risks(target_1, predict_1)

Прогноз для второго региона
Средняя прогнозируемая прибыль равна 0.45007168261029573
С вероятностью 95% прибыль будет от 0.044596630492171566 до 0.8307963015211544 млрд. руб.
риск убытка 1.0


In [21]:
print('Прогноз для третьего региона')
risks(target_2, predict_2)

Прогноз для третьего региона
Средняя прогнозируемая прибыль равна 0.39218018842906227
С вероятностью 95% прибыль будет от -0.14830850390371483 до 0.884150739869968 млрд. руб.
риск убытка 8.3


Самый низкий показатель риска убытка у второго региона. 

**Вывод: Мы выбрали модель Логистической регрессии и обучили ее на обучающей выборке для каждого региона. Сверив средние показатели запасов нефти в регионах определили что в третьем регионе среднее количиство запасов больше чем в остальных. Провели расчет предполагаемой прибыли и установили что во втором регионе она выше остальных. И сравнив вероятность убыточности выяснили что во втором регионе с вероятностью 95% разработки будут не убыточными от 0,045 до 0,83 млрд. руб. и риск убытка 1% что удовлетворяет поставленную задачу**